## Calculate (RESP) partial charges with Psi4
import the following modules

In [29]:
import os
import psi4
import resp
import openbabel as ob
from rdkit import Chem
from rdkit.Chem import AllChem

### some helper functions

In [30]:
def neutralize_atoms(mol):
    pattern = Chem.MolFromSmarts("[+1!h0!$([*]~[-1,-2,-3,-4]),-1!$([*]~[+1,+2,+3,+4])]")
    at_matches = mol.GetSubstructMatches(pattern)
    at_matches_list = [y[0] for y in at_matches]
    if len(at_matches_list) > 0:
        for at_idx in at_matches_list:
            atom = mol.GetAtomWithIdx(at_idx)
            chg = atom.GetFormalCharge()
            hcount = atom.GetTotalNumHs()
            atom.SetFormalCharge(0)
            atom.SetNumExplicitHs(hcount - chg)
            atom.UpdatePropertyCache()
    return mol

def cleanUp(psi4out_xyz):
    deleteTheseFiles = ['1_default_grid.dat','1_default_grid_esp.dat','grid.dat','timer.dat']
    deleteTheseFiles.append(psi4out_xyz)
    for fileName in deleteTheseFiles:
        if os.path.exists(fileName):
            os.remove(fileName)

def get_xyz_coords(mol):
    if not mol is None:
        num_atoms = mol.GetNumAtoms()
        xyz_string=""
        for counter in range(num_atoms):
            pos=mol.GetConformer().GetAtomPosition(counter)
            xyz_string = xyz_string + ("%s %12.6f %12.6f %12.6f\n" % (mol.GetAtomWithIdx(counter).GetSymbol(), pos.x, pos.y, pos.z) )
    return xyz_string


def calcRESPCharges(mol, basisSet, method, gridPsi4 = 1):
    options = {'BASIS_ESP': basisSet,
               'METHOD_ESP': method,
               'RESP_A': 0.0005,
               'RESP_B': 0.1,
               'VDW_SCALE_FACTORS':[1.4, 1.6, 1.8, 2.0],
               'VDW_POINT_DENSITY':int(gridPsi4)
    }

    resp_charges = resp.resp([mol], [options])[0][1]
    return resp_charges


### Set some variables and stuff

In [32]:
method = 'b3lyp'
basisSet = '3-21g' #'6-31G**'
neutralize = True
psi4.set_memory('10 GB')
obConversion = ob.OBConversion()
obConversion.SetInAndOutFormats("xyz", "mol2")
singlePoint = True
path = "./data"

### Read sdf file (3D) into a list 

In [24]:
inputFile = "./data/twoCpds.sdf"
molList = Chem.SDMolSupplier(inputFile, removeHs=False)

### ...or read a SMILES files into a list

In [38]:
SMILESasInput = False

if SMILESasInput:
    molList = []
    inputFile = "./data/twoCpds.smi"
    suppl = Chem.SmilesMolSupplier(inputFile, titleLine = False)

    for mol in suppl:
        mol = Chem.AddHs(mol)
        AllChem.EmbedMolecule(mol)
        try: 
            AllChem.MMFFOptimizeMolecule(mol)
        except: 
            AllChem.UFFOptimizeMolecule(mol)            
        molList.append(mol)

### Loop over compounds in list and calculate partial charges

In [39]:
for mol in molList:
    
    if not mol is None:

        molId = mol.GetProp("_Name")
        print('Trying:', molId)
        
        if neutralize:
            mol = neutralize_atoms(mol)
            mol = Chem.AddHs(mol)

        xyz_string = get_xyz_coords(mol) 
        psi_mol = psi4.geometry(xyz_string)

        ### single point calculation
        outfile_mol2 = inputFile[:-4]+".mol2"

        if singlePoint:
            print('Running singlepoint...')
            resp_charges = calcRESPCharges(psi_mol, basisSet, method, gridPsi4 = 1)

        else:
            print('Running geometry optimization...')            
            methodNbasisSet = method+"/"+basisSet
            psi4.optimize(methodNbasisSet, molecule=psi_mol)
            resp_charges = calcRESPCharges(psi_mol, basisSet, method, gridPsi4 = 1)            

        ### save coords to xyz file 
        psi4out_xyz = molId + '.xyz'
        psi_mol.save_xyz_file(psi4out_xyz,1)

        ### read xyz file and write as mol2 
        ob_mol = ob.OBMol()
        obConversion.ReadFile(ob_mol, psi4out_xyz)

        ### set new partial charges
        count = 0
        for atom in ob.OBMolAtomIter(ob_mol):
            newChg = resp_charges[count]
            atom.SetPartialCharge(newChg)
            count += 1

        ### write as mol2 
        outfile_mol2 = path+"/"+molId+"_partialChgs.mol2"
        print("Finished. Saved compound with partial charges as mol2 file: %s" % outfile_mol2)
        obConversion.WriteFile(ob_mol, outfile_mol2)

        ### clean up
        cleanUp(psi4out_xyz)


Trying: TXA
Running singlepoint...
Finished. Saved compound with partial charges as mol2 file: ./data/TXA_partialChgs.mol2
Trying: 4piol
Running singlepoint...
Finished. Saved compound with partial charges as mol2 file: ./data/4piol_partialChgs.mol2
